
# What is a neural network?

A neural network is a collection of **neurons** that are connected by layers. Each neuron is a small
computing unit that performs simple calculations to collectively solve a problem. Neurons are organized in three types of layers: input layer, hidden layer, and output layer. The hidden and output layers contain a number of neurons. Neural networks mimic the way a human brain processes information.

![Diagram showing the different types of layers in a neural network.](../images/4-model-1.png)


## Components of a neural network

- An **activation function** determines whether a neuron should be activated or not. The computations that happen in a neural network include applying an activation function. If a neuron activates, then it means the input is important. There are different kinds of activation functions. The choice of which activation function to use depends on what you want the output to be. Another important role of an activation function is to add non-linearity to the model.

    - _Binary_ is used to set an output node to 1 if the function result is positive and 0 if the function result is zero or negative. $f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ 1, & \text{if } x\geq 0\\ \end{cases}}$
    - _Sigmoid_ is used to predict the probability of an output node being between 0 and 1.  $f(x) = {\large \frac{1}{1+e^{-x}}} $
    - _Tanh_ is used to predict whether an output node is between 1 and -1, for classification use cases.  $f(x) = {\large \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}} $
    - _ReLU_ (*rectified linear activation function*) is used to set the output node to 0 if the function result is negative and keeps the result value if the result is a positive value.  $f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ x, & \text{if } x\geq 0\\ \end{cases}}$
- **Weights** influence how close our network's output is to the expected output value. As an input enters the neuron, it gets multiplied by a weight value, and the resulting output is either observed or passed to the next layer in the neural network. Weights for all neurons in a layer are organized into one tensor.
- **Bias** makes up the difference between the activation function's output and its intended output. A low bias suggests that the network is making more assumptions about the output's form, whereas a high bias value makes fewer assumptions about the output's form. 

![Diagram showing neural computation.](../images/4-model-2.png)

We can say that an output $y$ of a neural network layer with weights $W$ and bias $b$ is computed as summation of the inputs multiplied by the weights plus the bias. $x = \sum{(weights * inputs) + bias} $, where $f(x)$ is the activation function.


# Build a neural network

Neural networks are comprised of layers and modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses `nn.Module`. A neural network is itself a module that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.

In [14]:
%matplotlib inline
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get a hardware device for training

We want to be able to train our model on a hardware accelerator like a GPU, if one is available. Let's check to see whether 
`torch.cuda` is available; if not, we'll continue to use the CPU.

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


## Define the class

We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

Our neural network is composed of the following:

- The input layer with 28x28 or 784 features/pixels.
- The first linear module takes the input 784 features and transforms it to a hidden layer with 512 features.
- The ReLU activation function will be applied in the transformation.
- The second linear module takes 512 features as input from the first hidden layer and transforms it to the next hidden layer with 512 features.
- The ReLU activation function will be applied in the transformation.
- The third linear module take 512 features as input from the second hidden layer and transforms those features to the output layer with 10, which is the number of classes.
- The ReLU activation function will be applied in the transformation.

In [16]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, move it to the `device`, and print its structure.

In [17]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model's `forward`, along with some background operations. However, don't call `model.forward()` directly! Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.

We get the prediction densities by passing it through an instance of the `nn.Softmax`.

In [18]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4])


## Weight and Bias


The `nn.Linear` module randomly initializes the ${weights}$  and ${ bias}$ for each layer and internally stores the values in Tensors.

In [19]:
print(f"First Linear weights: {model.linear_relu_stack[0].weight} \n")

print(f"First Linear biases: {model.linear_relu_stack[0].bias} \n")

First Linear weights: Parameter containing:
tensor([[-0.0312, -0.0038,  0.0202,  ...,  0.0180,  0.0017,  0.0028],
        [ 0.0056, -0.0144, -0.0152,  ...,  0.0136, -0.0332, -0.0143],
        [ 0.0175, -0.0187,  0.0323,  ..., -0.0188,  0.0162,  0.0106],
        ...,
        [-0.0234,  0.0225,  0.0306,  ..., -0.0182,  0.0099,  0.0348],
        [-0.0249,  0.0259,  0.0150,  ..., -0.0179, -0.0284, -0.0284],
        [ 0.0287,  0.0155, -0.0023,  ...,  0.0058,  0.0344, -0.0125]],
       requires_grad=True) 

First Linear biases: Parameter containing:
tensor([ 2.7258e-02, -8.0004e-03,  2.7626e-02,  2.1964e-02,  4.4959e-03,
        -1.6973e-02, -3.1862e-02, -3.3186e-02,  2.6841e-03, -2.4953e-02,
        -8.7154e-03,  1.5384e-02, -2.7446e-02, -3.1534e-02,  2.7948e-02,
        -1.1457e-02, -1.1938e-02, -2.1420e-02, -5.0158e-03, -1.3121e-03,
         1.1885e-02,  1.1733e-02, -2.6689e-02,  1.8255e-02,  4.7913e-03,
        -1.7925e-04, -8.7950e-03,  2.0485e-02, -2.7262e-02, -1.7092e-02,
         2.3

## Model layers

Let's break down the layers in the FashionMNIST model. To illustrate it, we take a sample minibatch of three images of size **28x28** and see what happens to it as we pass it through the network. 

In [20]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

We initialize the `nn.Flatten` layer to convert each two-dimensional 28x28 image into a contiguous array of 784 pixel values; that is, the minibatch dimension (at dim=0) is maintained. Each of the pixels are passed to the neural network's input layer.  

![Diagram showing how pixels in an image are flattened.](../images/4-model-3.png)

In [21]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear 

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases. The grayscale value of each pixel in the input layer is connected to neurons in the hidden layer for calculation. The calculation the transformation uses is ${{weight * input + bias}} $.


In [22]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

Nonlinear activations are what create the complex mappings between the model's inputs and outputs. They're applied after linear transformations to introduce *nonlinearity*, helping neural networks learn a wide variety of phenomena. In this model, we use `nn.ReLU` between our linear layers, but there's other activations to introduce nonlinearity in your model.

The ReLU activation function takes the output from the linear layer calculation and replaces the negative values with zeros.

Linear output: ${ x = {weight * input + bias}} $.  
ReLU:  $f(x)= 
\begin{cases}
    0, & \text{if } x < 0\\
    x, & \text{if } x\geq 0\\
\end{cases}
$

In [23]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3741, -0.0748,  0.1123, -0.2522,  0.3893,  0.1552, -0.0401,  0.2228,
          0.3250,  0.4199, -0.0522,  0.4573,  0.4327, -0.7967,  0.0336,  0.2588,
          0.5921, -0.3618,  0.2669,  0.4841],
        [ 0.0205, -0.4341, -0.1817, -0.6690,  0.6846,  0.2329, -0.2395,  0.1810,
          0.0898,  0.2032, -0.0069,  0.6222,  0.4223, -0.9044, -0.1780, -0.0751,
          0.5968, -0.1573, -0.2048,  0.0226],
        [ 0.3153,  0.2154,  0.0551, -0.4598,  0.2289,  0.3959, -0.1911,  0.4055,
          0.1663,  0.2888, -0.1680,  0.1777,  0.4802, -0.9233, -0.2419,  0.2751,
          0.4596, -0.4224, -0.0535,  0.0303]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3741, 0.0000, 0.1123, 0.0000, 0.3893, 0.1552, 0.0000, 0.2228, 0.3250,
         0.4199, 0.0000, 0.4573, 0.4327, 0.0000, 0.0336, 0.2588, 0.5921, 0.0000,
         0.2669, 0.4841],
        [0.0205, 0.0000, 0.0000, 0.0000, 0.6846, 0.2329, 0.0000, 0.1810, 0.0898,
         0.2032, 0.0000, 0.6222, 0.4223, 0.0000, 0.00

### nn.Sequential

`nn.Sequential` is an ordered 
container of modules. The data is passed through all the modules in their defined order. You can use
sequential containers to put together a quick network like `seq_modules`.

In [24]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

The last linear layer of the neural network returns `logits` (the raw values in \[`-infty`, `infty`]), which are passed to the `nn.Softmax` module. The Softmax activation function calculates the probability of the output from the neural network. It's only used on a neural network's output layer. The results are scaled to values \[0, 1\], representing the model's predicted densities for each class. The `dim` parameter indicates the dimension along which the result values must sum to 1. The node with the highest probability predicts the desired output.

![Diagram showing the softmax formula.](../images/4-model-4.png)

In [25]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model parameters

Many layers inside a neural network are *parameterized*; that is, the layers have associated weights 
and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter and print its size and a preview of its values.

In [26]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0312, -0.0038,  0.0202,  ...,  0.0180,  0.0017,  0.0028],
        [ 0.0056, -0.0144, -0.0152,  ...,  0.0136, -0.0332, -0.0143]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0273, -0.0080], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0177,  0.0099,  0.0077,  ..., -0.0285,  0.0238,  0.0125],
        [ 0.0243,  0.0077,  0.0410,  ...,  0.0251, -0.0183,  0.0324]],
       grad_fn=<SliceBackward0>) 

Layer: linear_re